https://medium.com/@mohitdulani/fine-tune-any-llm-using-your-custom-dataset-f5e712eb6836 template by mohit dulani

In [1]:
!pip install -q torch dataset peft trl transformers bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    

In [2]:
import torch
from datasets import load_dataset
from google.colab import userdata, drive
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)

In [6]:
access_token = userdata.get("ACCESS_TOKEN")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
drive.mount('/content/drive')
%cd /content/drive/MyDrive/projects/finance-llm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/projects/finance-llm


In [14]:
model_name = "mistralai/Mistral-7B-V0.3"

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [9]:
peft_config = LoraConfig(
    r=16, lora_alpha=64, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    max_grad_norm=0.35,
    warmup_ratio=0.03,
    max_steps=100,
    save_steps=100,
    lr_scheduler_type="constant",
)

In [15]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    token=access_token
)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [27]:
dataset = load_dataset('json', data_files='output_formatted.jsonl', field='data', split='train')
dataset = dataset.train_test_split(test_size=0.1)

In [28]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 1207
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 135
    })
})


In [29]:
def createPrompt(example):
    bos_token = "<s>"
    system_prompt = "[INST] You are a finance suggestion model and your role is to give finance related suggestions \n"
    input_prompt = f" {example['input']} [/INST]"  # depends on the dataset
    output_prompt = f"{example['output']} </s>"

    return [bos_token + system_prompt + input_prompt + output_prompt]

In [30]:
trainer = SFTTrainer(
    model=model,
    peft_config=peft_config,
    formatting_func=createPrompt,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1207 [00:00<?, ? examples/s]

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [31]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [32]:
trainer.train()

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Step,Training Loss
10,0.770300
20,0.167600
30,0.030200
40,0.021400
50,0.019500
60,0.017900
70,0.015700
80,0.014800
90,0.014900
100,0.014000


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:619: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66d7aff7-346dc5c40b1412820074eab3;f8d1ce16-b369-4b90-bfe0-6137a7ac7ada)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-V0.3/resolve/main/config.json.
Access to model mistralai/Mistral-7B-v0.3 is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-V0.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:218: UserWarning: Could not find a config file in mistralai/Mistral-7B-V0.3 - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=100, training_loss=0.10863254189491273, metrics={'train_runtime': 325.5243, 'train_samples_per_second': 1.229, 'train_steps_per_second': 0.307, 'total_flos': 4374909891379200.0, 'train_loss': 0.10863254189491273, 'epoch': 100.0})

In [33]:
save_model = trainer.model.module if hasattr(trainer.model, "module") else trainer.model
save_model.save_pretrained("./results/model")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:619: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66d7aff7-0ca294f80523a05f4c3dbf27;b6478fc7-0eb7-4401-acd9-b612f83e56d4)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-V0.3/resolve/main/config.json.
Access to model mistralai/Mistral-7B-v0.3 is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-V0.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:218: UserWarning: Could not find a config file in mistralai/Mistral-7B-V0.3 - will assume that the vocabulary was not modified.
  warnings.warn(


In [34]:
lora_config = LoraConfig.from_pretrained("./results/model")
model = get_peft_model(model, lora_config)

In [ ]:
text = input(">>> ")
text = f"Question: {text}"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
inputs = tokenizer(text, return_tensors='pt').to(device)
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

>>> how to invest in mutual funds


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
